Diferente do processo de ETL, no ELT, os dados são carregados diretamente, da maneira que foram extraídos e a transformação é feita por comandos SQL, posteriormente.

In [8]:
import pandas as pd
from sqlalchemy import create_engine

# Extração dos dados e Load

In [9]:
# Configuração do Banco de Dados
engine = create_engine('sqlite:///emlurb_data.db')

arquivos = {
    '2022': 'data/156_2022.csv',
    '2024': 'data/156_2024.csv',
    '2025': 'data/156_2025.csv'
}

for ano, arquivo in arquivos.items():
    try:
        encoding = 'utf-8' if ano == '2022' else 'latin1'
        sep = ';'
        
        # Lê o CSV como texto, forçando tudo como string para evitar erros de tipo no load inicial
        df_raw = pd.read_csv(arquivo, sep=sep, encoding=encoding, dtype=str)
        
        df_raw['ano_origem'] = ano
        
        # 'if_exists="append"' vai empilhando os dados dos anos
        df_raw.to_sql('raw_chamados_156', con=engine, if_exists='append', index=False)
        
        print(f"Dados de {ano} carregados para a tabela 'raw_chamados_156'.")
        
    except Exception as e:
        print(f"Erro no ano {ano}: {e}")

Dados de 2022 carregados para a tabela 'raw_chamados_156'.
Dados de 2024 carregados para a tabela 'raw_chamados_156'.
Dados de 2025 carregados para a tabela 'raw_chamados_156'.


# Transformação dos dados

In [16]:
from sqlalchemy import text

# 1. Apagamos a View anterior que estava com erro
with engine.connect() as conn:
    conn.execute(text("DROP VIEW IF EXISTS view_chamados_tratados;"))
    print("View antiga removida.")

# 2. Recriamos a View, mas agora gerando o ID (ROW_NUMBER) e calculando a Duração
query_criacao_view_corrigida = """
CREATE VIEW view_chamados_tratados AS
SELECT
    -- GERA UM ID SEQUENCIAL AUTOMATICAMENTE
    ROW_NUMBER() OVER (ORDER BY DATA_DEMANDA) as _id,
    
    UPPER(GRUPOSERVICO_DESCRICAO) as gruposervico_descricao,
    UPPER(SERVICO_DESCRICAO) as servico_descricao,
    COALESCE(NULLIF(NUMERO, ''), 'S/N') as numero_tratado,
    COALESCE(NULLIF(LATITUDE, ''), 'INDETERMINADO') as latitude_tratada,
    DATA_DEMANDA,
    DATA_ULT_SITUACAO,
    
    -- Coluna RESOLVIDO
    CASE 
        WHEN SITUACAO = 'ATENDIDA' THEN 1 
        ELSE 0 
    END as resolvido,

    -- Coluna DURAÇÃO (Cálculo de dias no SQLite)
    -- JulianDay converte datas para números, permitindo subtração
    CASE 
        WHEN SITUACAO = 'ATENDIDA' THEN 
            CAST(julianday(DATA_ULT_SITUACAO) - julianday(DATA_DEMANDA) AS INTEGER)
        ELSE NULL 
    END as dias_decorridos

FROM raw_chamados_156;
"""

with engine.connect() as conn:
    conn.execute(text(query_criacao_view_corrigida))
    print("View corrigida criada com sucesso!")

View antiga removida.
View corrigida criada com sucesso!


In [17]:
df_analise = pd.read_sql("SELECT * FROM view_chamados_tratados LIMIT 20", engine)
display(df_analise)

,_id,gruposervico_descricao,servico_descricao,numero_tratado,latitude_tratada,DATA_DEMANDA,DATA_ULT_SITUACAO,resolvido,dias_decorridos
0,1,POSTE,IMPLANTACAO DE POSTE,7,INDETERMINADO,2008-03-14,2024-08-16,0,NaN
1,2,POSTE,IMPLANTACAO DE POSTE,7,INDETERMINADO,2008-03-14,2024-08-16,0,NaN
2,3,POSTE,IMPLANTACAO DE POSTE,50,INDETERMINADO,2008-04-10,2024-08-25,0,NaN
3,4,POSTE,IMPLANTACAO DE POSTE,50,INDETERMINADO,2008-04-10,2024-08-25,0,NaN
4,5,POSTE,IMPLANTACAO DE POSTE,00,INDETERMINADO,2008-08-15,2024-08-25,0,NaN
5,6,POSTE,IMPLANTACAO DE POSTE,00,INDETERMINADO,2008-08-15,2024-08-25,0,NaN
6,7,POSTE,IMPLANTACAO DE POSTE,200,INDETERMINADO,2008-09-15,2024-08-25,0,NaN
7,8,POSTE,IMPLANTACAO DE POSTE,200,INDETERMINADO,2008-09-15,2024-08-25,0,NaN
8,9,POSTE,IMPLANTACAO DE POSTE,09,INDETERMINADO,2009-03-10,2024-08-25,0,NaN
9,10,POSTE,IMPLANTACAO DE POSTE,09,INDETERMINADO,2009-03-10,2024-08-25,0,NaN
